In [1]:
#Calculates the number of Clenches in individual .csv file of EEG data caused by fluctuations in Electric Potential.

In [2]:
from scipy import fft, arange, signal
from scipy.special import logit, expit
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import os
import csv
import pandas as pd
from ttictoc import TicToc
#----------------------
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree
import xgboost
from sklearn.model_selection import train_test_split
#---------------------RF HP-f(x) & CV---------------
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score

In [3]:
EEG_FOLDER = "../../data/Clench/"

In [4]:
def read_file(path):
    """
    Reads the audio .wav file and returns the sample rate and data contents of the file.
    """
    sr, signal = wavfile.read(path)
    return sr, signal[:,0]

In [5]:
def read_all(files):
    """Branch audio file extrapolation. Uses read_file() method."""
    ret = []
    t = TicToc()
    t.tic();
    for fl in files:
        ret.append(read_file(fl))
    t.toc();
    print("Time it took to read data from files of length ", len(files), " = ", round(t.elapsed, 3), " seconds.")
    return np.array(ret)

In [6]:
def find_files(PATH, ext):
    """
    Finds all the files in a particular directory. Return only .csv files.
    """
    files = []
    for r, d, f in os.walk(PATH):
        for file in f:
            if ext in file:
                files.append(os.path.join(r, file).replace("\\","/"))
    return files

In [7]:
files = find_files(EEG_FOLDER, ".csv")

In [8]:
def remove_meta_data(PATH):
    """
    Return:
    1. Changes in Electric potential based on Unix timestamp from
        the 5 channels of the Emotiv headset. 2 channels from the Frontal Lobe, 
        1 channel from the parietal lobe, and 2 from temporal lobe.
    2. Pandas Dataframe of the data reflected from (1).
    """
    reader = csv.reader(open(PATH, "rt"), delimiter='\t')
    i = 0
    one_file_data = []
    for line in reader:
        if(i > 0):
            one_file_data.append(line)
        i += 1
    one_file_data = np.array(one_file_data)
    columns = one_file_data[0][0].split(",")[3:8]
    row_data = []
    for rows in one_file_data[1:]:
        dtx = rows[0].split(",")[3:8]
        cont = []
        for x in dtx:
            cont.append(float(x))
        row_data.append(cont)
    dataframe = pd.DataFrame(row_data, columns=columns)
    return np.array(row_data), dataframe

In [9]:
def data_DF_dir(list_PATH):
    """
    Returns all the data from a given set of path files and its associated pandas dataframe object.
    """
    raw_data = []
    dataframes = []
    for file in list_PATH:
        rd, dfob = remove_meta_data(file)
        raw_data.append(rd)
        dataframes.append(dfob)
    return raw_data, dataframes

In [10]:
bci_data, bci_df = data_DF_dir(files)

In [11]:
friendly_name = [x.split("/")[4].split("_")[0] for x in files]

In [67]:
def count_peaks_validation(peaks, x, cv_count):
    count = 0
    for states in peaks:
        if(states > cv_count):
            count += 1
#     plt.plot(x)
#     plt.xlabel("Time (sample_rate (128) per second expansion)")
#     plt.ylabel("Amplitude")
#     plt.title("Time Domain Chart of Peak count (One-Sided f(x))", y=1.08)
    return count

In [68]:
def ease(index, distance, start_index, cv_count):
    dataCV = []
    for i in range(0, 5):
        x = bci_data[index].T[i][start_index:]
        x = abs(x - np.average(x))  # biasing.
        peaks, props = signal.find_peaks(x, distance=distance)
        count = count_peaks_validation(peaks, x, cv_count)
        dataCV.append(count)
    count = round(np.array(dataCV).mean())
    return int(friendly_name[index]), count

In [69]:
def validation(distance, start_index, cv_count, CV=False):
    
    validation_score = 0
    samples_correct = []
    samples_wrong = []
    
    for index in range(len(friendly_name)):
        test, pred = ease(index, distance, start_index, cv_count)
        if(test == pred):
            validation_score += 1
            samples_correct.append(friendly_name[index])
        else:
            samples_wrong.append([friendly_name[index], pred])
    
    if(CV == False):
        print("Final Result :\n\nSamples Correct: ", str(validation_score), "\nSamples Incorrect : ", str(len(samples_wrong)), "\n")
        if(len(samples_wrong) > 0):
            for wr in range(len(samples_wrong)):
                print("Incorrect Sample #", str(wr+1), ": ", str(samples_wrong[wr][0]), "\tPredicted : ", str(samples_wrong[wr][1]))
    return validation_score, samples_correct, samples_wrong

In [82]:
high = []
start_index = 0
for sid in range(0,300):
    highest = 0
    high_index = 0
    count_first_off = False
    fluc_point = 0
    print("\n\n")
    for cv in range(200, 400):
        control = cv
        sc,_,_ = validation(control, sid, cv_count, True)
        if(sc > highest):
            highest = sc
            high_index = control
            start_index = sid
            if(highest >= 11):
                print("Highest score recorded at :", highest, "\t Index [distance] at : ", control, "\tStart Index :", sid)
        elif(sc < highest and highest >= 11 and count_first_off == False):
            print("Fluctuation from Highest Point at : ", control, "\tScore : ", sc)
            count_first_off = True
            fluc_point = control
    high.append((highest, high_index, fluc_point, sid))




Highest score recorded at : 11 	 Index [distance] at :  282 	Start Index : 0
Fluctuation from Highest Point at :  319 	Score :  10
Highest score recorded at : 12 	 Index [distance] at :  340 	Start Index : 0



Highest score recorded at : 11 	 Index [distance] at :  340 	Start Index : 1
Fluctuation from Highest Point at :  355 	Score :  9



Highest score recorded at : 11 	 Index [distance] at :  282 	Start Index : 2
Highest score recorded at : 12 	 Index [distance] at :  318 	Start Index : 2
Fluctuation from Highest Point at :  319 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  282 	Start Index : 3
Highest score recorded at : 12 	 Index [distance] at :  311 	Start Index : 3
Fluctuation from Highest Point at :  319 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  282 	Start Index : 4
Highest score recorded at : 12 	 Index [distance] at :  311 	Start Index : 4
Fluctuation from Highest Point at :  319 	Score :  11



Highest score record




Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 39
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 39
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 40
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 40
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 41
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 41
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 42
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 42
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 43
Highest score recorded at : 12 	 Index [distance] at :  250 	Start 




Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 78
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 78
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 79
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 79
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 80
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 80
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 81
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 81
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 82
Highest score recorded at : 12 	 Index [distance] at :  250 	Start 

Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 117
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 117
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 118
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 118
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 119
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 119
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 120
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 120
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 121
High




Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 155
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 155
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 156
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 156
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 157
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 157
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 158
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 158
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 159
Highest score recorded at : 12 	 Index [distance] at :  25




Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 194
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 194
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 195
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 195
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 196
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 196
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 197
Highest score recorded at : 12 	 Index [distance] at :  250 	Start Index : 197
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  249 	Start Index : 198
Highest score recorded at : 12 	 Index [distance] at :  25

Highest score recorded at : 12 	 Index [distance] at :  249 	Start Index : 232
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  217 	Start Index : 233
Highest score recorded at : 12 	 Index [distance] at :  249 	Start Index : 233
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  217 	Start Index : 234
Highest score recorded at : 12 	 Index [distance] at :  249 	Start Index : 234
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  217 	Start Index : 235
Highest score recorded at : 12 	 Index [distance] at :  249 	Start Index : 235
Fluctuation from Highest Point at :  255 	Score :  11



Highest score recorded at : 11 	 Index [distance] at :  217 	Start Index : 236
Highest score recorded at : 12 	 Index [distance] at :  249 	Start Index : 236
Fluctuation from Highest Point at :  255 	Score :  11



High

Fluctuation from Highest Point at :  225 	Score :  11
Highest score recorded at : 13 	 Index [distance] at :  249 	Start Index : 267



Highest score recorded at : 11 	 Index [distance] at :  215 	Start Index : 268
Highest score recorded at : 12 	 Index [distance] at :  223 	Start Index : 268
Fluctuation from Highest Point at :  225 	Score :  11
Highest score recorded at : 13 	 Index [distance] at :  249 	Start Index : 268



Highest score recorded at : 11 	 Index [distance] at :  215 	Start Index : 269
Highest score recorded at : 12 	 Index [distance] at :  223 	Start Index : 269
Fluctuation from Highest Point at :  225 	Score :  11
Highest score recorded at : 13 	 Index [distance] at :  249 	Start Index : 269



Highest score recorded at : 11 	 Index [distance] at :  215 	Start Index : 270
Highest score recorded at : 12 	 Index [distance] at :  223 	Start Index : 270
Fluctuation from Highest Point at :  225 	Score :  11
Highest score recorded at : 13 	 Index [distance] at :  249 	Sta

Highest score recorded at : 13 	 Index [distance] at :  249 	Start Index : 295
Fluctuation from Highest Point at :  255 	Score :  12



Highest score recorded at : 11 	 Index [distance] at :  214 	Start Index : 296
Highest score recorded at : 12 	 Index [distance] at :  217 	Start Index : 296
Highest score recorded at : 13 	 Index [distance] at :  249 	Start Index : 296
Fluctuation from Highest Point at :  255 	Score :  12



Highest score recorded at : 11 	 Index [distance] at :  214 	Start Index : 297
Highest score recorded at : 12 	 Index [distance] at :  217 	Start Index : 297
Highest score recorded at : 13 	 Index [distance] at :  249 	Start Index : 297
Fluctuation from Highest Point at :  255 	Score :  12



Highest score recorded at : 11 	 Index [distance] at :  214 	Start Index : 298
Highest score recorded at : 12 	 Index [distance] at :  217 	Start Index : 298
Highest score recorded at : 13 	 Index [distance] at :  249 	Start Index : 298
Fluctuation from Highest Point at :  25

In [83]:
high

[(12, 340, 319, 0),
 (11, 340, 355, 1),
 (12, 318, 319, 2),
 (12, 311, 319, 3),
 (12, 311, 319, 4),
 (12, 311, 319, 5),
 (12, 282, 319, 6),
 (12, 282, 319, 7),
 (12, 282, 319, 8),
 (12, 282, 319, 9),
 (12, 282, 301, 10),
 (12, 282, 301, 11),
 (12, 282, 301, 12),
 (12, 282, 290, 13),
 (12, 282, 290, 14),
 (12, 267, 255, 15),
 (12, 267, 255, 16),
 (12, 267, 255, 17),
 (12, 267, 255, 18),
 (12, 269, 255, 19),
 (12, 269, 255, 20),
 (12, 269, 255, 21),
 (12, 269, 255, 22),
 (12, 269, 255, 23),
 (12, 269, 255, 24),
 (12, 250, 255, 25),
 (12, 250, 255, 26),
 (12, 250, 255, 27),
 (12, 250, 255, 28),
 (12, 250, 255, 29),
 (12, 250, 255, 30),
 (12, 250, 255, 31),
 (12, 250, 255, 32),
 (12, 250, 255, 33),
 (12, 250, 255, 34),
 (12, 250, 255, 35),
 (12, 250, 255, 36),
 (12, 250, 255, 37),
 (12, 250, 255, 38),
 (12, 250, 255, 39),
 (12, 250, 255, 40),
 (12, 250, 255, 41),
 (12, 250, 255, 42),
 (12, 250, 255, 43),
 (12, 250, 255, 44),
 (12, 250, 255, 45),
 (12, 250, 255, 46),
 (12, 250, 255, 47),
 (